# 필요 라이브러리 임포트

In [53]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
import plotly.express as px
import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier

## 평가 산식 : Score = 0.6 × F1 + 0.4 × (1 − NMAE)
			
1) F1 = (2 × Precision × Recall) ÷ (Precision + Recall)

Precision = TP ÷ (TP + FP)  
Recall = TP ÷ (TP + FN)  

여기서  
TP(True Positive): 정답과 예측 모두에 포함된 공행성쌍  
FP(False Positive): 예측에는 있으나 정답에는 없는 쌍  
FN(False Negative): 정답에는 있으나 예측에 없는 쌍



2)

$$
\mathrm{NMAE}
= \frac{1}{|\mathcal{U}|}
\sum_{u \in \mathcal{U}}
\min\!\left(
1,\;
\frac{\left|y^{(u)}_{\text{true}} - y^{(u)}_{\text{pred}}\right|}
     {\left|y^{(u)}_{\text{true}}\right| + \epsilon}
\right)
$$

U = 정답 쌍(G)과 예측 쌍(P)의 합집합  
y_true: 정답의 다음달 무역량 (정수 변환)  
y_pred: 예측 무역량 (정수 반올림)  
FN 또는 FP에 해당하는 경우 오차 1.0(100%, 최하점)로 처리  
오차가 100%를 초과하는 경우에도 1.0(100%, 최하점)로 처리  

#### F1 score = 아이템의 고유 id 100개의 총 쌍의 갯수 9900개를 계산
예)  
1번 아이템과 3번 아이템이 공행성쌍일 경우 1, 3 pair는 true label,  
2번 아이템과 3번 아이템이 공행성쌍이 아닐 경우 2, 3 pair는 false label로 판별하여
주최측 기준으로 공행성쌍으로 판별한 아이템 pair를 찾아내야함

#### NMAE = MAE를 정규화하여 값의 단위가 다른 데이터셋에 적합한 평가 방식, 이 경우는 target 아이템의 지난 달, 이번 달 value와 pair 아이템의 이번 달 value로 target 아이템의 다음 달 value를 예측

다음 프레임은 실제 값 중 하나로 target 아이템 1과 pair 아이템 2로 칭함
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>아이템 1의 이번 달 value</th>
      <th>아이템 1의 지난 달 value</th>
      <th>아이템 2의 이번 달 value</th>
      <th>max_corr</th>
      <th>best_lag</th>
      <th>아이템 1의 다음 달 value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>38475.0</td>
      <td>12187.0</td>
      <td>14276.0</td>
      <td>0.383169</td>
      <td>2.0</td>
      <td>23209.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>23209.0</td>
      <td>38475.0</td>
      <td>52347.0</td>
      <td>0.383169</td>
      <td>2.0</td>
      <td>37804.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>37804.0</td>
      <td>23209.0</td>
      <td>53549.0</td>
      <td>0.383169</td>
      <td>2.0</td>
      <td>27145.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>27145.0</td>
      <td>37804.0</td>
      <td>0.0</td>
      <td>0.383169</td>
      <td>2.0</td>
      <td>1210.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1210.0</td>
      <td>27145.0</td>
      <td>26997.0</td>
      <td>0.383169</td>
      <td>2.0</td>
      <td>5943.0</td>
    </tr>
  </tbody>
</table>
</div>  
<br>
max_corr = 지정 지연달까지 가장 높게 나온 상관계수<br>
best_lag = 상관계수가 가장 높게 나오는 지연달<br>
'아이템 2의 이번 달'이라고 표기 했지만 실제로는 '아이템 1의 이번 달 - best_lag'의 값임<br>
이 경우는 아이템 1의 이번 달이 22년 3월이라고 할 경우 아이템 2는 22월 1월 value

<br><br>
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>시간</th>
      <th>아이템 2 value</th>
      <th>지연된 아이템 1 value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2022-01</th>
      <td>14276.0</td>
      <td>38475.0</td>
    </tr>
    <tr>
      <th>2022-02</th>
      <td>52347.0</td>
      <td>23209.0</td>
    </tr>
    <tr>
      <th>2022-03</th>
      <td>53549.0</td>
      <td>37804.0</td>
    </tr>
    <tr>
      <th>2022-04</th>
      <td>0.0</td>
      <td>27145.0</td>
    </tr>
    <tr>
      <th>2022-05</th>
      <td>26997.0</td>
      <td>1210.0</td>
    </tr>
  </tbody>
</table>
</div>
<br>
실제 값 비교 (이 경우 아이템 1은 2개월 뒤의 value부터 계산)

학습 데이터 로드

In [54]:
train = pd.read_csv('./train.csv')

각각 월별 무역량(value), 무게(weight)를 정리

In [55]:
# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)
monthly["ym"] = monthly["ym"].dt.strftime("%Y-%m")

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

pivot.head()

ym,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


In [56]:
# year, month, item_id 기준으로 weight 합산 (seq만 다르다면 weight 합산)
monthly_w = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly_w["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly_w["month"].astype(str).str.zfill(2)
)
monthly_w["ym"] = monthly_w["ym"].dt.strftime("%Y-%m")

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot_w = (
    monthly_w
    .pivot(index="item_id", columns="ym", values="weight")
    .fillna(0.0)
)

pivot_w.head()

ym,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,17625.0,67983.0,69544.0,0.0,34173.0,103666.0,0.0,0.0,0.0,0.0,...,786651.0,249144.0,33133.0,32937.0,33083.0,0.0,49050.0,0.0,865246.0,1046036.0
AHMDUILJ,100990.0,43444.0,64113.0,42637.0,21468.0,59424.0,61587.0,63625.0,61245.0,20382.0,...,42986.0,43763.0,24379.0,62351.0,23521.0,43332.0,44913.0,44035.0,25574.0,34463.0
ANWUJOKX,0.0,0.0,0.0,89967.0,118992.0,41649.0,13888.0,0.0,0.0,119940.0,...,0.0,0.0,0.0,37211.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,50193.0,81429.0,43310.0,62505.0,84680.0,37425.0,114600.0,39305.0,104865.0,43123.0,...,118952.0,698.0,0.0,1907.0,11.0,2777.0,347.0,335.0,4974.0,6314.0
ATLDMDBO,163308448.0,113468029.0,131798388.0,118641599.0,106301802.0,63769133.0,148292927.0,101468186.0,77986006.0,94320028.0,...,143545801.0,70368609.0,99495350.0,153804927.0,93762902.0,76888377.0,119375444.0,112349280.0,95457203.0,165713328.0


In [57]:
# 무역량이 발생한 달
np.sort((43 - (pivot.T == 0).sum()).unique())

array([ 1,  3,  5,  6,  7,  8, 10, 26, 27, 28, 29, 30, 31, 36, 37, 39, 40,
       41, 42, 43])

모든 아이템의 value를 시간순으로 시각화

In [58]:
df = pivot.T.reset_index()

lme_long = df.melt(id_vars="ym", var_name="Metal", value_name="value")


fig = px.line(
    lme_long,
    x="ym",
    y="value",
    color="Metal",
    labels={"date_id": "Date (date_id)", "value": ""}
)

# R의 figsize(16, 6) 비슷하게 크기 조정 (픽셀 단위)
fig.update_layout(width=1100, height=400, legend_title_text="Metal", margin=dict(l=40, r=20, t=60, b=40))

fig.show()

값의 차이가 매우 크기때문에 편한 시각화를 위해 MinMax 스케일링

In [59]:
df_v = pivot.T.reset_index()
df_w = pivot_w.T.reset_index()

scaler_v = MinMaxScaler()
scaler_w = MinMaxScaler()

df_v = pd.DataFrame(scaler_v.fit_transform(df_v.drop("ym",axis=1)),columns=df_v.drop("ym",axis=1).columns).set_index(df_v["ym"]).reset_index()
df_w = pd.DataFrame(scaler_w.fit_transform(df_w.drop("ym",axis=1)),columns=df_w.drop("ym",axis=1).columns).set_index(df_w["ym"]).reset_index()

아이템별 value 시각화

In [60]:
import plotly.express as px

uss_df = df_v.copy()

uss_long = uss_df.melt(id_vars="ym", var_name="item", value_name="value")

fig = px.line(
    uss_long,
    x="ym",
    y="value",
    color="item",
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,   # ★ 세로 간격 축소
    color_discrete_sequence=["#2962FF"],
)

# 패싯 라벨 정리 & 범례 제거 & 레이아웃 크기 조정
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width=1200, height=1400, showlegend=False)

# 주의: y축을 0~150으로 제한해 대부분의 티커 가독성을 높였습니다. 고가 티커는 전체 범위가 보이지 않을 수 있습니다
fig.show()

아이템별 value를 weight를 시각화

In [61]:
# 1) 길게 만들기 + 출처(series) 라벨 부여
long_a = (
    df_v
      .melt(id_vars="ym", var_name="item", value_name="value")
      .assign(series="value")
)
long_b = (
    df_w
      .melt(id_vars="ym", var_name="item", value_name="value")
      .assign(series="weight")
)

# 2) (선택) 공통 item만 사용하고 싶다면 inner merge 대신 교집합 필터
common_items = sorted(set(long_a["item"]).intersection(set(long_b["item"])))
long_a = long_a[long_a["item"].isin(common_items)]
long_b = long_b[long_b["item"].isin(common_items)]

# 3) 합치기
plot_df = pd.concat([long_a, long_b], ignore_index=True)

# 4) 라인 플롯: 패싯은 item, 색상/스타일은 series(두 개 라인)
fig = px.line(
    plot_df,
    x="ym",
    y="value",
    color="series",        
    line_dash="series", 
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,
    color_discrete_map={
        "value": "#2962FF",
        "weight": "#3B3A3A",
    },
    category_orders={"series": ["value", "weight"]},
)

# 5) 패싯 제목 간소화(=뒤 텍스트만 남기기) + 레이아웃
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width=1200, height=1400, legend_title_text="Series", showlegend=True)

fig.update_traces(hovertemplate="ym=%{x}<br>%{legendgroup}: %{y:.2f}")

fig.show()

In [62]:
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, max_lag=6, min_nonzero=12, corr_threshold=0.4):
    '''
    pivot = 상관관계를 비교할 column 리스트
    max_lag = 지연일
    min_nonzero = 무역량이 존재하는 달의 최소 수치
    corr_threshold = 상관계수 임계값
    '''
    items = pivot.index.to_list()                           # item_id 값 추출
    months = pivot.columns.to_list()                        # 시간 정보 추출
    n_months = len(months)                                  # 시간 정보 최대 길이
    results = []                                            # 결과 저장 용 빈 리스트
    
    for i, leader in tqdm(enumerate(items)):                # 단일 item_id 별 반복
        x = pivot.loc[leader].values.astype(float)          # 단일 item 시간 별 무역량
        if np.count_nonzero(x) < min_nonzero:               # 무역량이 존재하는 달의 총합 수가 지정한 최소 수치(min_nonzero)를 넘는 지 판별
            continue

        for follower in items:                              # target_item을 제외한 다른 item과 corr 비교
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)    # 비교할 item의 무역량 최소 수치 판별
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None                                 # corr이 제일 높게 나오는 지연일
            best_corr = 0.0                                 # 제일 높은 corr

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):               # 1일 차이부터 max_lag까지 corr 비교
                if n_months <= lag:                         # 아마도 max_lag 값 오입력 할 경우 예외 처리
                    print("이게 통과하는 경우가 있어?")
                    continue
                corr = safe_corr(x[:-lag], y[lag:]) 
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)                           # 반환을 위한 데이터프레임화
    return pairs

In [63]:
def find_comovement_multi_pairs(pivot_v, pivot_w, max_lag=6, min_nonzero=12, corr_threshold=0.4):
    '''
    pivot_v = value의 상관관계를 비교할 column 리스트
    pivot_w = weight의 상관관계를 비교할 column 리스트
    max_lag = 지연일
    min_nonzero = 무역량이 존재하는 달의 최소 수치
    corr_threshold = 상관계수 임계값
    '''
    items = pivot_v.index.to_list()                         # item_id 값 추출
    months = pivot_v.columns.to_list()                      # 시간 정보 추출
    n_months = len(months)                                  # 시간 정보 최대 길이
    results = []                                            # 결과 저장 용 빈 리스트
    
    for i, leader in tqdm(enumerate(items)):                # 단일 item_id 별 반복
        x = pivot_v.loc[leader].values.astype(float)          # 단일 item 시간 별 무역량
        x_w = pivot_w.loc[leader].values.astype(float) 
        if np.count_nonzero(x) < min_nonzero:               # 무역량이 존재하는 달의 총합 수가 지정한 최소 수치(min_nonzero)를 넘는 지 판별
            continue

        for follower in items:                              # target_item을 제외한 다른 item과 corr 비교
            if follower == leader:
                continue

            y = pivot_v.loc[follower].values.astype(float)    # 비교할 item의 무역량 최소 수치 판별
            y_w = pivot_w.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None                                 # corr이 제일 높게 나오는 지연일
            best_corr = 0.0                                 # 제일 높은 corr

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):               # 1일 차이부터 max_lag까지 corr 비교
                if n_months <= lag:                         # 아마도 max_lag 값 오입력 할 경우 예외 처리
                    print("이게 통과하는 경우가 있어?")
                    continue
                corr_v = safe_corr(x[:-lag], y[lag:])                                   # value의 상관계수
                corr_w = safe_corr(x_w[:-lag], y_w[lag:])                               # weight의 상관계수
                corr = max(abs(corr_v), abs(corr_w), (abs(corr_v) + abs(corr_w))/2)
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)                           # 반환을 위한 데이터프레임화
    return pairs

In [64]:
pairs = find_comovement_pairs(pivot, max_lag=6, corr_threshold=0.35)
print("탐색된 공행성쌍 수:", len(pairs))
pairs.head()

100it [00:06, 14.36it/s]

탐색된 공행성쌍 수: 2178


,leading_item_id,following_item_id,best_lag,max_corr
0,AANGBULD,APQGTRMF,5,-0.443984
1,AANGBULD,DDEXPPXU,2,0.383169
2,AANGBULD,DEWLVASR,6,0.640221
3,AANGBULD,DNMPSKTB,4,-0.410635
4,AANGBULD,EVBVXETX,6,0.436623


In [65]:
# pair가 하나라도 존재하는 아이템
pairs.leading_item_id.unique()

array(['AANGBULD', 'AHMDUILJ', 'APQGTRMF', 'ATLDMDBO', 'AXULOHBQ',
       'BEZYMBBT', 'BJALXPFS', 'BLANHGYY', 'BSRMSVTC', 'BTMOEMEP',
       'BUZIIBYG', 'CCLHWFWF', 'DBWLZWNK', 'DDEXPPXU', 'DEWLVASR',
       'DJBLNPNC', 'DNMPSKTB', 'DUCMGGNW', 'ELQGMQWE', 'EVBVXETX',
       'FCYBOAXC', 'FDXPMYGF', 'FITUEHWN', 'FQCLOEXA', 'FRHNWLNI',
       'FTSVTTSR', 'FWUCPMMW', 'GKQIJYDH', 'GYHKIVQT', 'HCDTGMST',
       'HXYSSRXE', 'IGDVVKUD', 'JBVHSUWY', 'JERHKLYW', 'JPBRUTWP',
       'JSLXRQOK', 'KAGJCHMR', 'KEUWZRKO', 'KJNSOAHR', 'LLHREMKS',
       'LPHPPJUG', 'LRVGFDFM', 'LSOIUSXD', 'LTOYKIML', 'LUENUFGA',
       'MBSBZBXA', 'MIRCVAMV', 'NAQIHUKZ', 'NZKBIBNU', 'OGAFEHLU',
       'OJIFIHMZ', 'OKMBFVKS', 'OXKURKXR', 'PYZMVUWD', 'QJQJSWFU',
       'QKXNTIIB', 'QRKRBYJL', 'QVLMOEYE', 'RAWUKQMJ', 'RCBZUSIM',
       'RJGPVEXX', 'ROACSLMG', 'SAAYMURU', 'SAHWCZNH', 'SDWAYPIK',
       'SNHYOVBM', 'STZDBITS', 'SUOYXCHP', 'TGOELCAG', 'UGEQLMXM',
       'UIFPPCLR', 'UQYUIVVR', 'UXSPKBJR', 'VBYCLTYZ', 'VMAQST

In [66]:
# 아이템 별 pair 갯수 -> 페어 item id를 워한다면 len() 제거
pairs_count = {}
for i in pairs.leading_item_id.unique():
    pairs_count[i] = len(pairs[pairs["leading_item_id"] == i]["following_item_id"].values)
pairs_count # 페어

{'AANGBULD': 26,
 'AHMDUILJ': 25,
 'APQGTRMF': 35,
 'ATLDMDBO': 40,
 'AXULOHBQ': 30,
 'BEZYMBBT': 26,
 'BJALXPFS': 25,
 'BLANHGYY': 28,
 'BSRMSVTC': 26,
 'BTMOEMEP': 39,
 'BUZIIBYG': 14,
 'CCLHWFWF': 24,
 'DBWLZWNK': 37,
 'DDEXPPXU': 9,
 'DEWLVASR': 29,
 'DJBLNPNC': 17,
 'DNMPSKTB': 46,
 'DUCMGGNW': 9,
 'ELQGMQWE': 30,
 'EVBVXETX': 30,
 'FCYBOAXC': 10,
 'FDXPMYGF': 26,
 'FITUEHWN': 8,
 'FQCLOEXA': 29,
 'FRHNWLNI': 26,
 'FTSVTTSR': 18,
 'FWUCPMMW': 17,
 'GKQIJYDH': 9,
 'GYHKIVQT': 42,
 'HCDTGMST': 18,
 'HXYSSRXE': 36,
 'IGDVVKUD': 30,
 'JBVHSUWY': 27,
 'JERHKLYW': 12,
 'JPBRUTWP': 45,
 'JSLXRQOK': 13,
 'KAGJCHMR': 19,
 'KEUWZRKO': 16,
 'KJNSOAHR': 19,
 'LLHREMKS': 18,
 'LPHPPJUG': 27,
 'LRVGFDFM': 35,
 'LSOIUSXD': 27,
 'LTOYKIML': 16,
 'LUENUFGA': 17,
 'MBSBZBXA': 10,
 'MIRCVAMV': 14,
 'NAQIHUKZ': 15,
 'NZKBIBNU': 28,
 'OGAFEHLU': 39,
 'OJIFIHMZ': 10,
 'OKMBFVKS': 39,
 'OXKURKXR': 36,
 'PYZMVUWD': 22,
 'QJQJSWFU': 19,
 'QKXNTIIB': 13,
 'QRKRBYJL': 41,
 'QVLMOEYE': 36,
 'RAWUKQMJ': 24,
 

기준 아이템과 공행성쌍으로 판별된 아이템의 실제 value와 비교

In [67]:
# 1) 기준 아이템 설정
REF_ITEM = "ZKENOUDA"   # 예: pairs.leading_item_id.unique() 중 하나

target_col = np.append(["ym",REF_ITEM],pairs[pairs["leading_item_id"] == REF_ITEM]["following_item_id"].values)

uss_df = df_v[target_col].copy()

# pivot_longer → melt
uss_long = uss_df.melt(id_vars="ym", var_name="item", value_name="value")

# 2) 기준 시계열 추출
bench = (
    uss_long.loc[uss_long["item"] == REF_ITEM, ["ym", "value"]]
            .rename(columns={"value": "bench_value"})
)

# 3) 모든 아이템에 기준값 머지
dfc = uss_long.merge(bench, on="ym", how="left")

# 4) '자기 자신' 라인과 '기준' 라인 두 벌로 쌓기
plot_df = pd.concat(
    [
        dfc.assign(series="self",  val=dfc["value"]),
        dfc.assign(series=f"benchmark: {REF_ITEM}", val=dfc["bench_value"])
    ],
    ignore_index=True
)

# 5) 그리기: 패싯은 item으로, 색은 series로
fig = px.line(
    plot_df,
    x="ym",
    y="val",
    color="series",
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,
    # 원하시면 색을 고정해 가독성을 높일 수 있습니다.
    color_discrete_map={
        "self": "#2962FF",                 # 각 아이템(자기 자신)
        f"benchmark: {REF_ITEM}": "#9E9E9E"  # 기준 라인(회색)
    }
)

# 패싯 라벨 정리 & 범례/레이아웃
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_traces(opacity=0.95)
fig.update_layout(width=1400, height=700, showlegend=True)

# 필요하면 Y축 범위 고정(주의: 기준과 스케일이 다르면 왜곡될 수 있음)
# fig.update_yaxes(range=[0, 150])

fig.show()

In [68]:
def build_training_data(pivot, pairs):
    """
    공행성쌍 + 시계열을 이용해 (X, y) 학습 데이터를 만드는 함수
    input X:
      - b_t, b_t_1, a_t_lag, max_corr, best_lag
    target y:
      - b_t_plus_1
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "target": b_t_plus_1,
            })

    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot, pairs)
print('생성된 학습 데이터의 shape :', df_train_model.shape)
df_train_model.head()

생성된 학습 데이터의 shape : (83732, 6)


,b_t,b_t_1,a_t_lag,max_corr,best_lag,target
0,582317.0,539873.0,14276.0,-0.443984,5.0,759980.0
1,759980.0,582317.0,52347.0,-0.443984,5.0,216019.0
2,216019.0,759980.0,53549.0,-0.443984,5.0,537693.0
3,537693.0,216019.0,0.0,-0.443984,5.0,205326.0
4,205326.0,537693.0,26997.0,-0.443984,5.0,169440.0


In [69]:
test_df = pivot.T[["AANGBULD"]].iloc[:-5]
test_df["APQGTRMF_laged"] = pivot.T["APQGTRMF"].iloc[5:].values
test_df

item_id,AANGBULD,APQGTRMF_laged
ym,,
2022-01,14276.0,582317.0
2022-02,52347.0,759980.0
2022-03,53549.0,216019.0
2022-04,0.0,537693.0
2022-05,26997.0,205326.0
2022-06,84489.0,169440.0
2022-07,0.0,698033.0
2022-08,0.0,367604.0
2022-09,0.0,216414.0
